In [1]:
from theano.sandbox import cuda
cuda.use('gpu2')

Using gpu device 2: GeForce GTX TITAN X (CNMeM is enabled with initial size: 90.0% of memory, cuDNN 4007)


In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [3]:
#path = "data/ml-20m/"
path = "data/ml-small/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size=64

## Set up data

In [4]:
ratings = pd.read_csv(path+'ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [32]:
len(ratings)

100004

In [6]:
users = ratings.userId.unique()
movies = ratings.movieId.unique()

In [7]:
userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

In [8]:
ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])

In [9]:
user_min, user_max, movie_min, movie_max = (ratings.userId.min(), 
    ratings.userId.max(), ratings.movieId.min(), ratings.movieId.max())
user_min, user_max, movie_min, movie_max

(0, 670, 0, 9065)

In [10]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
n_users, n_movies

(671, 9066)

In [11]:
n_factors = 50

In [12]:
np.random.seed = 42

In [13]:
msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]

## Create subset for Excel

In [76]:
g=ratings.groupby('userId')['rating'].count()

In [77]:
topUsers=g.sort_values(ascending=False)[:15]

In [78]:
g=ratings.groupby('movieId')['rating'].count()

In [79]:
topMovies=g.sort_values(ascending=False)[:15]

In [80]:
top_r = ratings.join(topUsers, rsuffix='_r', how='inner', on='userId')

In [81]:
top_r = top_r.join(topMovies, rsuffix='_r', how='inner', on='movieId')

In [82]:
pd.crosstab(top_r.userId, top_r.movieId, top_r.rating, aggfunc=np.sum)

movieId,27,49,57,72,79,89,92,99,143,179,180,197,402,417,505
userId,,,,,,,,,,,,,,,
14,3.0,5.0,1.0,3.0,4.0,4.0,5.0,2.0,5.0,5.0,4.0,5.0,5.0,2.0,5.0
29,5.0,5.0,5.0,4.0,5.0,4.0,4.0,5.0,4.0,4.0,5.0,5.0,3.0,4.0,5.0
72,4.0,5.0,5.0,4.0,5.0,3.0,4.5,5.0,4.5,5.0,5.0,5.0,4.5,5.0,4.0
211,5.0,4.0,4.0,3.0,5.0,3.0,4.0,4.5,4.0,NaN,3.0,3.0,5.0,3.0,NaN
212,2.5,NaN,2.0,5.0,NaN,4.0,2.5,NaN,5.0,5.0,3.0,3.0,4.0,3.0,2.0
293,3.0,NaN,4.0,4.0,4.0,3.0,NaN,3.0,4.0,4.0,4.5,4.0,4.5,4.0,NaN
310,3.0,3.0,5.0,4.5,5.0,4.5,2.0,4.5,4.0,3.0,4.5,4.5,4.0,3.0,4.0
379,5.0,5.0,5.0,4.0,NaN,4.0,5.0,4.0,4.0,4.0,NaN,3.0,5.0,4.0,4.0
451,4.0,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,4.0,4.0,2.0,3.5,5.0


## Dot product

In [14]:
user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users, n_factors, input_length=1, W_regularizer=l2(1e-5))(user_in)
movie_in = Input(shape=(1,), dtype='int64', name='movie_in')
m = Embedding(n_movies, n_factors, input_length=1, W_regularizer=l2(1e-5))(movie_in)

In [197]:
x = merge([u, m], mode='dot')
x = Flatten()(x)
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse')

In [198]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=6, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80232 samples, validate on 19772 samples
Epoch 1/6
80232/80232 [==============================] - 1s - loss: 9.9858 - val_loss: 3.7782
Epoch 2/6
80232/80232 [==============================] - 1s - loss: 2.3523 - val_loss: 1.8406
Epoch 3/6
80232/80232 [==============================] - 1s - loss: 1.3676 - val_loss: 1.4714
Epoch 4/6
80232/80232 [==============================] - 1s - loss: 1.0832 - val_loss: 1.3430
Epoch 5/6
80232/80232 [==============================] - 1s - loss: 0.9504 - val_loss: 1.2896
Epoch 6/6
80232/80232 [==============================] - 1s - loss: 0.8712 - val_loss: 1.2714


##  Bias

In [15]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg))(inp)

In [223]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-5)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-5)

In [227]:
def create_bias(inp, n_in):
    return Flatten()(Embedding(n_in, 1, input_length=1)(inp))

In [228]:
ub = create_bias(user_in, n_users)
mb = create_bias(movie_in, n_movies)

In [229]:
x = merge([u, m], mode='dot')
x = Flatten()(x)
x = merge([x, ub], mode='sum')
x = merge([x, mb], mode='sum')
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse')

In [230]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=6, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80232 samples, validate on 19772 samples
Epoch 1/6
80232/80232 [==============================] - 2s - loss: 2.1477 - val_loss: 1.6632
Epoch 2/6
80232/80232 [==============================] - 2s - loss: 1.1876 - val_loss: 1.2984
Epoch 3/6
80232/80232 [==============================] - 2s - loss: 0.9035 - val_loss: 1.1801
Epoch 4/6
80232/80232 [==============================] - 2s - loss: 0.7640 - val_loss: 1.1356
Epoch 5/6
80232/80232 [==============================] - 2s - loss: 0.6778 - val_loss: 1.1219
Epoch 6/6
80232/80232 [==============================] - 2s - loss: 0.6127 - val_loss: 1.1185


##  Neural net

In [27]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-5)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-5)

In [28]:
x = merge([u, m], mode='concat')
x = Flatten()(x)
x = Dense(100, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1)(x)
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse')

In [29]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=5, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 79749 samples, validate on 20255 samples
Epoch 1/5
79749/79749 [==============================] - 2s - loss: 1.7501 - val_loss: 0.8424
Epoch 2/5
79749/79749 [==============================] - 2s - loss: 0.9993 - val_loss: 0.8322
Epoch 3/5
79749/79749 [==============================] - 2s - loss: 0.9286 - val_loss: 0.8064
Epoch 4/5
79749/79749 [==============================] - 2s - loss: 0.8656 - val_loss: 0.8113
Epoch 5/5
79749/79749 [==============================] - 2s - loss: 0.8182 - val_loss: 0.7959
